# Python Basics. Final Project. The Game.

Andrey Baranov. HSE University, Faculty of Computer Science

## "Dungeon and Pythons" game. The idea.

The game represents a genre of simple fighting or 2D action. Gamefield is an imaginary dungeon in which the Hero and the Monster spawn on the left and right edges of a screen, respectively. Both hero and monster have stats, such as HP (a quantitative measure of "life"), damage (amount of the opponent's HP to be subtracted after the successful attack). Also, the hero has a defense number. It means when the hero is in defense positions, the amount of this parameter would be subtracted from the monster damage to the hero. If it's more, than monster damage, nothing happens, and the monster can't do any damage to the hero.

The main purpose is to kill a monster, meaning do damage to it successfully enough times to make it zero. And in the process, to not have such damage yourself. The one whose HP will become zero first is considered killed. In that case, for the hero, the game is lost; otherwise, the player has won the game.

In pseudo-random order, the bottles of three different colors (red, blue, and yellow) are scattered across the gamefield. In the order that no bottle at the start of a game will collide with a monster or hero. Each bottle could be collected by the hero if he has touched it and has enough capacity left. The capacity is three bottles for the hero inventory. The hero cannot have any more bottles in the inventory while the old ones are still there.

Bottles could be used by the hero to increase certain parameters, such as HP, damage, and defense numbers. Red for HP, blue for damage, and yellow for defense. When specified in the game help button is pressed on the keyboard, the effect will take place, and the bottle will be removed from the inventory and deleted from the current game session. The bottle effect is considered constant in terms of the current game, so the hero parameters would only increase after the use of bottles.

The basic parameters for the hero and the monster will change according to the difficulty level of the current game. The exclusion is the HP amount for both. But the hero could still get more than 100 HP after using the red "health" bottles. The game has three different difficulty levels that can be chosen from the main menu. When the difficulty level increases, the damage and defense parameters for the hero decrease, and the opposite is true for the monster.

There are two classes for the hero type: warrior and archer. The warrior could only attack when faced with the monster. The archer can make distant attacks with the arrows. Also, they have different skins. The monster is always the same and can only attack. It tries to get closer to the position of the hero. And when it collides, it starts to attack automatically while this condition is held.

## Implementation

The game program uses the pygame module, version 2.4.0. It provides a graphical interface, draws objects on the screen, and has many ways to represent the game objects and their interactions. The implementation is based on the object-oriented programming paradigm. There are a few classes implementing the game itself, hero, monster, bottles, arrows, buttons and controls, and finally game settings.

In [14]:
# install specified package
%pip install pygame==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# import needed libraries
import pygame
from pygame.sprite import Sprite
import sys
import random

### Hero class

The first class is the one for the Hero. It has methods to initialize and show the hero on the screen. Attack the monster and defend from it. To move across the game screen. Also, methods to collect bottles on the gamefield, draw them in inventory on the screen, and use them

In [16]:
class Hero(Sprite):
    """Hero class"""

    def __init__(self, dnp_game, hero_class='warrior') -> None:
        """
            Initialize base parameters for the hero
        """
        super().__init__()
        self.screen = dnp_game.screen
        self.screen_rect = dnp_game.screen.get_rect()
        self.settings = dnp_game.settings

        #  Load the hero image and get its rect.
        if hero_class == 'warrior':
            self.basic_image = pygame.image.load('images/hero.png')
            self.image = self.basic_image
            self.attack_image = pygame.image.load('images/hero_attacks.png')
            self.defense_image = pygame.image.load('images/hero_defend.png')
            self.lost_image = pygame.image.load('images/hero_lost.png')
        elif hero_class == 'archer':
            self.basic_image = pygame.image.load('images/archer.png')
            self.image = self.basic_image
            self.attack_image = pygame.image.load('images/archer_attacks.png')
            self.defense_image = pygame.image.load('images/archer_defend.png')
            self.lost_image = pygame.image.load('images/archer_lost.png')

        # self.image = pygame.transform.scale(self.image, (60, 60))
        self.rect = self.image.get_rect()

        #  Start new hero at the left of the screen
        self.rect.midleft = self.screen_rect.midleft

        #  Store a decimal value for the hero's speed
        self.x = float(self.rect.x)
        self.y = float(self.rect.y)

        # movement and defending flag
        self.moving_right = False
        self.moving_left = False
        self.moving_up = False
        self.moving_down = False
        self.defending = False

        # hero inventory bottles
        self.hero_bottles = pygame.sprite.Group()
        self.bottles_max = 3
        self.hero_class = hero_class

    def blitme(self):
        """
            Draw the hero at its current location.
        """
        self.screen.blit(self.image, self.rect)

    def update(self):
        """
            Update the hero's position based on movement flags.
        """
        if self.moving_right and self.rect.right < self.screen_rect.right:
            self.x += self.settings.hero_speed
        elif self.moving_left and self.rect.left > 0:
            self.x -= self.settings.hero_speed
        elif self.moving_up and self.rect.top > 0:
            self.y -= self.settings.hero_speed
        elif self.moving_down and self.rect.bottom < self.screen_rect.bottom:
            self.y += self.settings.hero_speed

        #  Update rect
        self.rect.x = self.x
        self.rect.y = self.y

    def attack(self, monster):
        """
            When monster's sprite is collided
            Does damage to the monster if hero is warrior
            Else just changes the hero image
        """
        # change image
        self.image = self.attack_image
        if self.hero_class == 'warrior':
            # check if the monster nearby, calculate damage
            if self.rect.colliderect(monster) and monster.settings.monster_hp > 0:
                monster.settings.monster_hp -= self.settings.hero_damage
            if monster.settings.monster_hp < 0:
                monster.settings.monster_hp = 0

    def attack_finished(self):
        """
            Finishes attack
            Return basic image
        """
        self.image = self.basic_image

    def defense(self):
        """
            Set defending flag
            And changes image
        """
        self.image = self.defense_image
        # only when defending defense points are applicable
        self.defending = True

    def defense_finished(self):
        """
            Back to the normal image and False flag
            when the defense is finished
        """
        self.image = self.basic_image
        self.defending = False

    def check_bottle_col(self, dnp_game):
        """
            Checks if hero collided with any bottle
            Adds it to the hero inventory if it doesn't surpass max capacity
        """
        if len(self.hero_bottles) < self.bottles_max \
                and pygame.sprite.spritecollideany(self, dnp_game.bottles):
            collisions = pygame.sprite.spritecollide(self, dnp_game.bottles, True)
            bottle_color = collisions.pop().color

            hero_bottle = Bottle(self, bottle_color, size=(20, 30))
            self.hero_bottles.add(hero_bottle)
            self.draw_hero_bottles(dnp_game)

    def draw_hero_bottles(self, dnp_game):
        """
            Draws hero inventory bottles
        """
        count = 1
        for bottle in self.hero_bottles:
            bottle.rect.top = dnp_game.screen.get_rect().top
            bottle.rect.x = 3 + (count * bottle.rect.width + 1)
            count += 1
        self.hero_bottles.draw(dnp_game.screen)

    def use_bottle(self, color):
        """
            If specific bottle used
            It affects some parameters
            Afterward bottle is removed from screen
        """
        for bottle in self.hero_bottles:
            if bottle.color == color:
                if color == 'red':
                    self.settings.hero_hp += self.settings.hp_effect
                elif color == 'blue':
                    self.settings.hero_damage += self.settings.damage_effect
                elif color == 'yellow':
                    self.settings.hero_defense += self.settings.defense_effect
                self.hero_bottles.remove(bottle)
                break

### Monster class

The second class implements Monster. It is almost identical to the hero one but has a few differences. Monster cannot defend, collect, or use bottles to increase its stats. It could only spawn, move, and attack the hero if they collided. The last two actions are happening automatically.

In [17]:
class Monster(Sprite):
    def __init__(self, dnp_game) -> None:
        """
            Initialize monster basic parameters
        """
        super().__init__()
        self.screen = dnp_game.screen
        self.screen_rect = dnp_game.screen.get_rect()
        self.settings = dnp_game.settings

        #  Load the image and get its rect.
        self.basic_image = pygame.image.load('images/snake.png')
        self.image = self.basic_image
        self.attack_image = pygame.image.load('images/snake_attacks.png')
        self.lost_image = pygame.image.load('images/snake_lost.png')
        self.attacking = False

        # self.image = pygame.transform.scale(self.image, (60, 60))
        self.rect = self.image.get_rect()

        #  Start each new monster at the right of the screen
        self.rect.midright = self.screen_rect.midright

        #  Store a decimal value for the monster's speed
        self.x = float(self.rect.x)
        self.y = float(self.rect.y)

    def attack(self, hero):
        """
            Monster attack method
            It can only attack with the specific damage
        """
        self.image = self.attack_image
        # calculate damage
        if hero.settings.hero_hp <= 0:
            return
        if hero.defending:
            if hero.settings.hero_defense < self.settings.monster_damage:
                hero.settings.hero_hp -= (self.settings.monster_damage - hero.settings.hero_defense)
        else:
            hero.settings.hero_hp -= self.settings.monster_damage
        if hero.settings.hero_hp < 0:
            hero.settings.hero_hp = 0

    def attack_finished(self):
        """
            Return to the basic image when attack finishes
        """
        self.image = self.basic_image

    def update(self, dnp_game):
        """
            Updates monster positions
            Makes it move in direction of the hero
            with the specific speed
        """
        hero_x = dnp_game.hero.rect.x
        hero_y = dnp_game.hero.rect.y
        mnst_speed = dnp_game.settings.monster_speed

        # check case when hero is near
        # attack!
        if self.rect.colliderect(dnp_game.hero):
            self.attack(dnp_game.hero)
            # screen update
            dnp_game.update_screen()
            pygame.time.delay(200)
            self.attack_finished()
        else:
            if self.x < hero_x:
                self.x += mnst_speed
            elif self.x > hero_x:
                self.x -= mnst_speed
            if self.y < hero_y:
                self.y += mnst_speed
            elif self.y > hero_y:
                self.y -= mnst_speed

        #  Update rect
        self.rect.x = self.x
        self.rect.y = self.y

    def blitme(self):
        """
            Draw the monster at its current location.
        """
        self.screen.blit(self.image, self.rect)

### Bottle class

This class represents bottles of different colors and properties. Each color affects different hero stats. e.g., red for the HP, blue for the damage, and yellow for the defense.

In [18]:
class Bottle(Sprite):
    """
        Bottles that could be collected
        Placed on the game field or in the inventory
        Affect different hero parameters
    """

    def __init__(self, dnp_game, color_name='red', size=(40, 60)) -> None:
        """
            initializes a bottle of a color with a size
        """
        super().__init__()
        self.screen = dnp_game.screen
        self.settings = dnp_game.settings
        self.color = color_name

        self.image = pygame.image.load(f'images/{color_name}_bottle.png')
        self.image = pygame.transform.scale(self.image, size)
        self.rect = self.image.get_rect()

### Arrow class

The arrow class is used to create, show, and move arrow objects. Arrows could only be made by the archer hero class. It is the standard distance attack type for this hero's choice. The warrior doesn't have access to this. The arrow could only move from left to right on the gamefield. It's a powerful weapon, so the idea here was not to make it too strong.

In [19]:
class Arrow(Sprite):
    """
        Manages arrow object at the archer current position
        For archer hero class only
    """

    def __init__(self, dnp_game):
        """
            Initialize the arrow and sets basic parameters
        """
        super().__init__()
        self.screen = dnp_game.screen
        self.settings = dnp_game.settings
        # self.color = self.settings.arrow_color
        self.image = pygame.image.load('images/arrow.png')
        self.rect = self.image.get_rect()

        # create an arrow rect at (0, 0) and set correct position
        self.rect = pygame.Rect(0, 0, self.settings.arrow_width,
                                self.settings.arrow_height)
        self.rect.midleft = dnp_game.hero.rect.midright
        self.rect.y -= 15

        #  Store arrow's position
        self.x = float(self.rect.x)

### Settings class

This is a utility class. It is used to store most of the number values in one place, trying to eliminate so called "magic numbers". Also, it's changing hero and monster stats dynamically according to the chosen difficulty level.

In [20]:
class Settings:
    """
        Game settings class
        Mainly to eliminate "magic numbers"
    """

    def __init__(self) -> None:
        # None will be set in dynamic settings
        self.monster_speed = None
        self.monster_damage = None
        self.hero_defense = None
        self.hero_damage = None
        self.monster_hp = None
        self.hero_hp = None
        self.hero_speed = None
        # game settings
        self.screen_width = 1200
        self.screen_height = 800
        self.bg_color = (230, 230, 230)
        self.difficulty = "easy"
        # inventory settings
        self.max_inv = 3
        # bottle settings
        self.hp_effect = 20
        self.damage_effect = 5
        self.defense_effect = 5
        # arrow settings
        self.arrow_width = 15
        self.arrow_height = 3
        self.arrow_color = (60, 60, 60)
        self.arrows_allowed = 3
        self.arrow_speed = 3.0
        self.initialize_dynamic_settings()

    def initialize_dynamic_settings(self):
        """
            Dynamic settings for different difficulty
            And also refreshes HP for hero/monster for the new session
        """
        # hero settings
        self.hero_speed = 4
        self.hero_hp = 100
        # monster settings
        self.monster_hp = 100

        """change settings according to the difficulty"""
        if self.difficulty == "easy":
            self.hero_damage = 10
            self.hero_defense = 5
            self.monster_damage = 3
            self.monster_speed = 1
        elif self.difficulty == "medium":
            self.hero_damage = 7
            self.hero_defense = 3
            self.monster_damage = 5
            self.monster_speed = 1.5
        elif self.difficulty == "hard":
            self.hero_damage = 5
            self.hero_defense = 2
            self.monster_damage = 7
            self.monster_speed = 2


### Buttons classes

Here are not one, but three classes. The basic button class for any button on the screen. In the case of the current game, it is the Play button. Difficulty button to choose a different difficulty level. and the Hero button to choose the hero class (warrior or archer) for the current game session.

In [23]:
class Button:
    """
        Clickable buttons class
    """

    def __init__(self, dnp_game, msg):
        """
            initialize attributes for button
        """
        self.screen = dnp_game.screen
        self.screen_rect = self.screen.get_rect()

        #  Set the dimensions and properties of the button
        self.width, self.height = 200, 50
        self.button_color = (0, 0, 0)
        self.text_color = (192, 192, 192)
        self.font = pygame.font.SysFont(None, 48)

        #  Build the button's rect object and center it
        self.rect = pygame.Rect(0, 0, self.width, self.height)
        self.rect.center = self.screen_rect.center

        #  The button message
        self._prep_msg(msg)

    def _prep_msg(self, msg):
        """
            Turn msg into a rendered image and center text on the button
        """
        self.msg_image = self.font.render(msg, True, self.text_color,
                                          self.button_color)
        self.msg_image_rect = self.msg_image.get_rect()
        self.msg_image_rect.center = self.rect.center

    def draw_button(self):
        """
            Draw a blank button and then draw message
        """
        self.screen.fill(self.button_color, self.rect)
        self.screen.blit(self.msg_image, self.msg_image_rect)


class DifficultyButton(Button):
    """Class for the specific button for the difficulty"""

    def __init__(self, dnp_game, msg, location):
        super().__init__(dnp_game, msg)
        if location == "topright":
            self.rect.topright = self.screen_rect.topright
        elif location == "midright":
            self.rect.midright = self.screen_rect.midright
        elif location == "bottomright":
            self.rect.bottomright = self.screen_rect.bottomright
            # by the stats font up
            self.rect.y -= 24
        self._prep_msg(msg)


class HeroButton(Button):
    """Specific hero class choice button"""

    def __init__(self, dnp_game, msg, location):
        super().__init__(dnp_game, msg)
        if location == 'midleft':
            self.rect.midleft = self.screen_rect.midleft
        elif location == 'bottomleft':
            self.rect.bottomleft = self.screen_rect.bottomleft
            # by the stats font up
            self.rect.y -= 24
        self._prep_msg(msg)

### Game class. DungeonAndPythons


The class for the game object itself. It is the main class in the program. It initiates all the parameters for the game, creates all objects, starts the game, and holds the main game loop. It also draws and shows objects on the screen, checks if some of them have collided, and processes them accordingly. Also, it tracks the current game state if it hasn't started yet or finished already. Print the welcome screen with all the buttons and options, but also the final win or lose message. Basically, it is the control center for the whole game. The one that holds all the other classes together.

In [24]:
class DungeonAndPythons:
    """Manage a game"""

    def __init__(self) -> None:
        """
        Initialize a game. Creating Hero and Monster
        And other parameters
        """
        pygame.init()
        self.settings = Settings()
        self.screen = pygame.display.set_mode(
            (self.settings.screen_width, self.settings.screen_height))
        pygame.display.set_caption("Dungeon'n'Pythons")
        self.fpsClock = pygame.time.Clock()
        self.hero_class = 'warrior'

        self.hero = Hero(self, self.hero_class)
        self.monster = Monster(self)
        self.bottles = pygame.sprite.Group()
        self.arrows = pygame.sprite.Group()

        self.text_color = (30, 30, 30)
        self.font = pygame.font.SysFont(None, 48)
        self.font_stats = pygame.font.SysFont(None, 24)
        self.font_help = pygame.font.SysFont(None, 14)

        #  Make the play button
        self.play_button = Button(self, "Play ('p')")

        #  Make difficulty buttons
        self.easy_button = DifficultyButton(self, "Easy", "topright")
        self.medium_button = DifficultyButton(self, "Medium", "midright")
        self.hard_button = DifficultyButton(self, "Hard", "bottomright")

        # Hero buttons
        self.warrior_button = HeroButton(self, "Warrior", "midleft")
        self.archer_button = HeroButton(self, "Archer", "bottomleft")

        self.game_active = False

    def _create_bottles(self):
        """
            Creates and places bottles which could be collected
            by the hero. Bottles affect hp, damage and defense parameters
        """
        for color in {'red', 'yellow', 'blue'}:
            for _ in range(3):
                bottle = Bottle(self, color)
                max_x = self.screen.get_rect().width - bottle.rect.width
                max_y = self.screen.get_rect().height - bottle.rect.height

                bottle.rect.x = random.randint(0, max_x)
                bottle.rect.y = random.randint(0, max_y)

                #  start each new alien near random place of the screen
                while bottle.rect.colliderect(self.hero) \
                        or bottle.rect.colliderect(self.monster) \
                        or pygame.sprite.spritecollideany(bottle, self.bottles):
                    bottle.rect.x = random.randint(0, max_x)
                    bottle.rect.y = random.randint(0, max_y)

                self.bottles.add(bottle)

    def _start_game(self):
        """
           Initialize the basic game parameters
           Cleans the screen from previous game session
        """
        self.settings.initialize_dynamic_settings()
        self.game_active = True
        self.bottles.empty()
        self.arrows.empty()
        self._create_bottles()
        self.hero = Hero(self, self.hero_class)
        self.monster = Monster(self)
        pygame.mouse.set_visible(False)
        self.run_game()

    def run_game(self):
        """
            Manages main loop for the game
        """
        while True:
            # 60 fps lock
            self.fpsClock.tick(60)
            self._check_events()
            if self.game_active:
                self.hero.update()
                self.monster.update(self)
                self.arrows.update()
                self._update_arrows()
                hp_check = self._check_hp()
                if hp_check is None:
                    pass
                else:
                    self.game_active = False
                    pygame.mouse.set_visible(True)
            self.update_screen()

    def _check_events(self):
        """
            Respond to key presses and mouse events.
        """
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                self._check_play_button(mouse_pos)
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_play_button(self, mouse_pos):
        """
            Process mouse clicks on buttons
        """
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if not self.game_active:
            if button_clicked:
                self._start_game()
            elif self.easy_button.rect.collidepoint(mouse_pos):
                self.settings.difficulty = "easy"
            elif self.medium_button.rect.collidepoint(mouse_pos):
                self.settings.difficulty = "medium"
            elif self.hard_button.rect.collidepoint(mouse_pos):
                self.settings.difficulty = "hard"
            elif self.warrior_button.rect.collidepoint(mouse_pos):
                self.hero_class = 'warrior'
            elif self.archer_button.rect.collidepoint(mouse_pos):
                self.hero_class = 'archer'

    def _check_keydown_events(self, event):
        """
            Process keyboard keys interaction
        """
        if not self.game_active and event.key == pygame.K_p:
            self._start_game()
        elif event.key == pygame.K_RIGHT:
            self.hero.moving_right = True
        elif event.key == pygame.K_LEFT:
            self.hero.moving_left = True
        elif event.key == pygame.K_UP:
            self.hero.moving_up = True
        elif event.key == pygame.K_DOWN:
            self.hero.moving_down = True
        elif event.key == pygame.K_q:
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self.hero.attack(self.monster)
            if self.hero_class == 'archer':
                self._fire_arrow()
        elif event.key == pygame.K_LCTRL:
            self.hero.defense()
        elif event.key == pygame.K_h:
            # health bottle
            self.hero.use_bottle('red')
        elif event.key == pygame.K_a:
            # damage blue
            self.hero.use_bottle('blue')
        elif event.key == pygame.K_d:
            # defense yellow
            self.hero.use_bottle('yellow')

    def _check_keyup_events(self, event):
        """
            Process finish of pressing keyboard keys
        """
        if event.key == pygame.K_RIGHT:
            self.hero.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.hero.moving_left = False
        elif event.key == pygame.K_UP:
            self.hero.moving_up = False
        elif event.key == pygame.K_DOWN:
            self.hero.moving_down = False
        elif event.key == pygame.K_SPACE:
            self.hero.attack_finished()
        elif event.key == pygame.K_LCTRL:
            self.hero.defense_finished()

    def update_screen(self):
        """
            Update images on the screen, and flip to the new screen.
        """
        self.screen.fill(self.settings.bg_color)
        self.hero.blitme()
        self.monster.blitme()

        # checks hp of hero/monsters to display message
        self._check_hp()

        self._check_hero()
        self._check_monster()
        self._render_help()
        for arrow in self.arrows.sprites():
            arrow.draw_arrow()

        self.bottles.draw(self.screen)
        self.hero.check_bottle_col(self)
        self.hero.draw_hero_bottles(self)

        #  Draw buttons if game is inactive
        if not self.game_active:
            self.play_button.draw_button()
            self.easy_button.draw_button()
            self.medium_button.draw_button()
            self.hard_button.draw_button()
            self.warrior_button.draw_button()
            self.archer_button.draw_button()

        #  Make the most recently drawn screen visible.
        pygame.display.flip()

    def _check_hp(self):
        """
            Checks the remaining hero/monster HP
            If <=0, game finishes
        """
        self.message = ''
        if self.settings.monster_hp <= 0:
            self.message = 'YOU WON!'
            self.monster.image = self.monster.lost_image
        elif self.settings.hero_hp <= 0:
            self.message = 'YOU LOST...'
            self.hero.image = self.hero.lost_image
        if self.message:
            self.final_message = self.font.render(self.message, True,
                                                  self.text_color, self.settings.bg_color)
            self.message_rect = self.final_message.get_rect()
            self.message_rect.center = self.screen.get_rect().center
            self.message_rect.y += 50
            self.screen.blit(self.final_message, self.message_rect)
            return True
        return None

    def _check_hero(self):
        """
            Check and display hero current stats
        """
        text = 'HERO '
        text += f'HP:{self.settings.hero_hp} '
        text += f'Damage:{self.settings.hero_damage} '
        text += f'Defense:{self.settings.hero_defense} '

        message = self.font_stats.render(text, True,
                                         self.text_color, self.settings.bg_color)
        message_rect = message.get_rect()
        message_rect.bottomleft = self.screen.get_rect().bottomleft
        self.screen.blit(message, message_rect)

    def _check_monster(self):
        """
            Check and display monster current stats
        """
        text = 'MONSTER '
        text += f'HP:{self.settings.monster_hp} '
        text += f'Damage:{self.settings.monster_damage} '
        # text += f'Defense: {self.settings}\n'

        message = self.font_stats.render(text, True,
                                         self.text_color, self.settings.bg_color)
        message_rect = message.get_rect()
        message_rect.bottomright = self.screen.get_rect().bottomright
        self.screen.blit(message, message_rect)

    def _render_help(self):
        """
            Renders help for control keys
        """
        help_text = [
            '(SPACE) for attack',
            '(LCtrl) for defense',
            '(H) for health (red) bottle',
            '(D) for defense (yellow) bottle',
            '(A) for damage (blue) bottle',
            '(Q) for exit'
        ]
        count = 0
        for line in help_text:
            message = self.font_help.render(line, True,
                                            self.text_color, self.settings.bg_color)
            message_rect = message.get_rect()
            message_rect.topleft = self.screen.get_rect().topleft
            message_rect.y += 31 + count * 14
            count += 1
            self.screen.blit(message, message_rect)

    def _fire_arrow(self):
        """
            Creates new arrow and add it to the group
        """
        if len(self.arrows) < self.settings.arrows_allowed:
            new_arrow = Arrow(self)
            self.arrows.add(new_arrow)

    def _update_arrows(self):
        """
            Update position and delete old arrows
        """
        self.arrows.update()

        for arrow in self.arrows.copy():
            if arrow.rect.right > self.screen.get_rect().right:
                self.arrows.remove(arrow)

        self._check_bullet_monster_collision()

    def _check_bullet_monster_collision(self):
        """
            Check collision between any arrow and monster
            Does damage to the monster and deletes arrow
        """
        if pygame.sprite.spritecollideany(self.monster, self.arrows):
            collisions = pygame.sprite.spritecollide(self.monster, self.arrows, True)
            self.arrows.remove(collisions.pop())
            self.monster.settings.monster_hp -= self.hero.settings.hero_damage
            if self.monster.settings.monster_hp < 0:
                self.monster.settings.monster_hp = 0

### Other info about classes

Many features of this game are based on the classes and methods provided by the PyGame library. For example, hero, monster, and even the arrows or bottles objects are inherited from Sprite class of the PyGame module. This way, all the work to represent it on the screen and check the collisions is up to the module itself. And the work here includes only developing the program logic itself, writing a set of class methods, and processing interactions between different parts of the game. In terms of the current final project conditions specified in the evaluation criteria, the classes methods represent the implementation of a functions, which are needed to successfully pass the final project check.

## Game controls


When the game starts, it first shows the welcome screen. The welcome screen has a few buttons: three to choose difficulty level (easy, medium, and hard), two to choose hero class (warrior or archer), and one to start the game (Play button), which could also be started by pressing the 'P' key on the keyboard. Other buttons take effect when they are clicked by the mouse cursor. The last button that was clicked is considered the final choice for a certain option (difficulty or hero class).

After the play button is clicked or the 'P' key is pressed, the monster will start to move across the gamefield in the direction of the hero in order to attack him. The hero could be controlled by arrow keys on the keyboard in any of four directions from edge to edge of the screen. Also, the hero automatically collects up to three unused bottles, which could be used by the specified keys: 'H' - for the health bottle (red), 'A' - for the damage bottle (blue), and 'D' - for the defense bottle (yellow). The space bar is used to attack the monster by the hero. Warrior class could only do damage to the monster if they collided. Archer in turn, could make up to three arrows on the gamefield at the same time. Arrows moving from the left to the right of the screen do damage to the monster if they collide. Arrows would disappear in that case, or when reaching the far right border of the screen. When the arrow disappeared, the archer could shoot another one.

When the hero's or monster's HP is zero, the game ends with the player's win or loss. The player could start a new session by clicking the Play button or pressing 'P' keys, that way, all the parameters will be reset and the game will start clean. Also, at the end of the game session, any parameter could be changed, such as hero class or difficulty. In any case, the key 'Q' could be used to exit the game window.

## Program example

Due to limitations of either Google Colab or a local Jupyter notebook. The example of the game described above is provided in a separate file called 'final_proj_branov.py'. In order for this game to work, it is extremely necessary to have the PyGame module installed; version 2.4.0 is recommended. Also, the images for the game object sprites must be inside the "images" catalog in the same directory as the main .py file of the program. For this project, I'll try to attach the current .ipynb, .py  file and images of the game program in the form of .zip archive or/and separately. To execute the program, it is enough to execute 'python ./final_proj_baranov.py' command in the terminal or the command line in the graphical interface of the OS (e.g. Windows).